In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

from IPython.display import Markdown
def bold(string):
    display(Markdown(string))
    
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

In [2]:
train = pd.read_csv(r'.\Data\problem_train.csv')
test = pd.read_csv(r'.\Data\problem_test.csv')

labels = pd.read_csv(r'.\Data\problem_labels.csv')

In [3]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)
labels = labels.drop('id', axis=1)


train_nas = train.iloc[:, list(train.isnull().sum() / len(train) * 100 == 100)].columns
test_nas = test.iloc[:, list(test.isnull().sum() / len(test) * 100 == 100)].columns
nas_cols = list(set(train_nas) | set(test_nas))

train = train.drop(nas_cols, axis=1)
test = test.drop(nas_cols, axis=1)


train_dropped = []
for col in train.columns:
    if (train[col].isnull().sum() <= 1000) & (len(train[col].value_counts()) == 1) or \
        (train[col].isnull().sum() == train.shape[0]):
            train_dropped.append(col)
            train = train.drop(col, axis=1)
            
test_dropped = []
for col in test.columns:
    if (test[col].isnull().sum() <= 1000) & (len(test[col].value_counts()) == 1) or \
        (test[col].isnull().sum() == test.shape[0]):
            test_dropped.append(col)
            test = test.drop(col, axis=1)
            
            
cols_to_drop = []
for col in train.columns:
    if len(train[col].value_counts()) == 1:
        if train[col].value_counts()[train[col].value_counts().keys()[0]] == train.shape[0]:
            cols_to_drop.append(col)
print(cols_to_drop)

cols_to_drop = []
for col in test.columns:
    if len(test[col].value_counts()) == 1:
        if test[col].value_counts()[test[col].value_counts().keys()[0]] == test.shape[0]:
            cols_to_drop.append(col)
print(cols_to_drop)
            
sorted(train.columns) == sorted(test.columns)

[]
[]


True

In [4]:
train_noNas = list(train.iloc[:, list(train.isnull().any() == False)].columns)
test_noNas = list(test.iloc[:, list(test.isnull().any() == False)].columns)
noNAs = list(set(train_noNas) & set(test_noNas))

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

---

## Gaps filling: part IV

На самом деле, восстанавливать плотности была не очень здравой идей изначально, но время было, и мне было интересно.

И, вообще говоря, еще до этого помнил про такую штуку, как SVD, отложил и благополучно забыл, а теперь опять вспомнил. Какое счастье, а ведь чуть не забыл окончательно!

---

### Singular value decomposition

**Теорема (SVD)**:
<br>Произвольная $l \times n$-матрица представима в виде $$F = VDU^{T}$$
<br>где 
* $l \times n$-матрица $V = (v_1, \ldots, v_n)$ ортонормирована $\left( V^{T} V = E_n \right)$, 
<br>столбцы $v_j$ - собственные векторы матрицы $F F^{T}$;
* $n \times n$-матрицы $D = diag \left( \sqrt{\lambda_n}, \ldots,\sqrt{\lambda_n} \right)$,
<br>$\lambda_j \geq 0$ - собственные значения матриц $F F^{T}$ и $F^{T} F$;
* $n \times n$-матрица $U = (u_1, \ldots, u_n)$ ортонормирована $\left( Г^{T} Г = E_n \right)$, 
<br>столбцы $u_j$ - собственные векторы матрицы $F^{T} F$.

<br>---

<br>Иногда по разным причинам мы можем хотеть обратить матрицу $F$ (например, при поиске решения множественной линейной регрессии методом МНК. Но обратная не всегда хорошо ищется. Например, у нас могу быть линейно-зависимые столбцы, и тогда обратная матрица вообще не считается, либо у нас могут быть коррелированные столбцы или интразинзитивно коррелированы ($\rho(A, B), \ \rho(B, C) > 0$, но $\rho(A, C) < 0$).

Если у нас есть почти линейно зависимые столбцы в матрице $F$, то некоторые с.зн. (собственные значения) матрицы $F F^{T} \rightarrow 0$. Если размах максимального и минимального с.зн. составляет порядки, то возникает вычислительная неустойчивость, решение будет плохо интерпретируемым, а также возникнет эффект переобучения (на обучении метрика классная, а на контроле сасная).

Это можно побороть, например, через введение регуляризации, которая вместе с этим будет понижать нам размерность, тем самым отбирать признаки.

<br>---

<br>**Principal component analysis**:
<br>Но от мультиколлинеарности можно избавляться не только с помощью регуляризации, но и переходом от имеющихся признаков к каким-то новым. Т.е. произойдет изменение признаково пространства, скорее всего, с понижением его размерности.

$f_1(x), \ldots, f_n(x)$ - исходные *числовые* признаки;
<br>$g_1(x), \ldots, g_m(x)$ - новые *числовые* признаки, $m \leq n$
<br>$+$ *требование*: старые признаки должны линейно восстанавливаться по новым как можно точнее на обучающейся выборке $x_1, \ldots, x_l$: 
$$\hat{f_j}(x) = \sum\limits_{s=1}^{m} g_s(x) u_{js}, \ \forall x \in X; \\
\sum\limits_{i=1}^{l} \sum\limits_{j=1}^{n} \left( \hat{f_j}(x_i) - f_j(x_i) \right)^{2} \rightarrow \min\limits_{\{g_s(x_i)\}, \{u_{js}\} }$$
Т.е. хотим так подобрать и новые признаки, и их линейные комбинации, чтобы старые признаки восстанавливались как можно более надежно относительно выбранного функционала качества.

<br>Введенем понятие *матричной нормы Фробениуса* - сумма квадратов всех элементов матрицы. Будем искать преобразование матрицы в новое подпространство относительно данной нормы: $F \in M_{l, n}, \ G \in M_{l, m}, \ m \leq n; \quad U \in M_{n, m}$.
<br>Хотим: $\hat{F} = G U^{T} \approx F$, где $G$ - новые признаки, $U$ - преобразование.

<br>---

<br>**Теорема (PCA)**:
<br>Если $m \leq rk F$, то минимум $\left\lVert G U^{T} - F \right\rVert^{2}$ достигается, когда столбцы $U$ - с.в. матрицы $F^{T} F$, соответствующие $m$ максимальным с.зн. $\lambda_1, \ldots, \lambda_m$, <br>а матрица $G = FU$. При этом:
* матрица $U$ ортонормирована $\left( U^{T} U = E_m \right)$;
* матрица G ортогональна: $G^{T} G = \Lambda = diag(\lambda_1, \ldots, \lambda_m)$;
* $U \Lambda = F^{T} F U; \quad G \Lambda = F F^{T} G$;
* $\left\lVert G U^{T} - F \right\rVert^{2} = \left\lVert F \right\rVert^{2} - tr \Lambda = \sum\limits_{i=m+1}^{n} \lambda_i$.

*Суть*: если возьмем все с.зн. матрицы $F^{T} F$, отсортируем их по убыванию, то первые $m$ из них будут определять наше решение, а последние - ошибку описания.

<br>Если $m = n$, то 
* $\left\lVert G U^{T} - F \right\rVert^{2} = 0$;
* представление $\hat{F} = G U^{T} = F$ точное и совпаддает с SVD при $G = V \sqrt{\Lambda}$: $\ F = V \sqrt{\Lambda} U^{T}$;
* линейное преобразование $U$ работает в обе стороны: $F = G U^{T}; \quad G = F U$.

Поскольку новые признаки некоррелированы $\left( G^{T} G = \Lambda \right)$, преобразование $U$ называется *декоррелирующим* (или преобразование *Каруне-Лоэва*).

<br>---

<br>Будем искать главные компоненты $u_1, \ldots, u_n$, которые удовлетворяют следующим требованиям:
* ортогональность: $\langle u_i, u_j \rangle = 0, \ i != j$;
* нормированность: $\left\lVert u_i \right\rVert^{2} = 1$;
* при проецировании выборки на компоненты $u_1, \ldots, u_m$ получается максимальная дисперсия среди всех возможных способов выбрать $m$ компонент.

Чтобы понизить размерность выборки до $m$, будем проецировать ее на первые $m$ компонент -- из последнего свойства следует, что это оптимальный способ снижения размерности. Дисперсия проецированной выборку будет показывать, как много информации удалось сохранить после понижения размерности -- поэтому требуем максимальной дисперсии от проекций.

Проекция объекта $x_k$ на компоненту $u_i$ вычисляется как $\langle x_k , u_i \rangle$, а проекция всей выборки на эту компоненту - как $X u_i$. Если за $U_m$ обозначить матрицу, столбцы которой равны первым $m$ компонентам, проекция выборки на них будет записываться как $X U_m$, а дисперсия проецированной выборки будет вычисляться как след ковариационной матрицы: $tr U_{m}^{T} X^{T} X U_{m} = \sum\limits_{i=1}^{m} \left\lVert X u_i \right\rVert^{2}.

Начнем с первой компоненты. Сведем все требования к ней в оптимизационную задачу:
$$\left\lVert X u_1 \right\rVert^{2} \rightarrow \max\limits_{u_1} \quad \& \quad
\left\lVert u_1 \right\rVert^{2} = 1$$
Запишем лагранжиан: 
$$L(u_1, \lambda) = \left\lVert X u_1 \right\rVert^{2} + \lambda \left( \left\lVert u_1 \right\rVert^{2} - 1 \right)$$
Продифференцируем и приравняем к нулю:
$$\frac{\partial L}{\partial u_1} = 2 X^{T} X u_1 + 2 \lambda u_1 = 0$$
Отсюда получаем, что $u_1$ должен быть собственным вектором ковариационной матрицы $X^{T} X$. Учтем это и преобразуем функционал:
$$ \left\lVert X u_1 \right\rVert^{2} = u_{1}^{T} X^{T} X u_{1} = \lambda u_{1}^{T} u_{1} = \lambda \rightarrow \max\limits_{u_1}$$
Значит, собственный вектор $u_1$ должен соответствовать максимальному с.зн.

Для следующих компонент к оптимизационной задаче будут добавляться требования ортогональности предыдущим компонентам. Решая эти задачи, мы получим, что главная компонента $u_i$ равна с.в., соответствующему $i$-му с.зн.
<br>После того, как найдены главные компоненты, можно проецировать на них данные. При таком способе новые признаки вычисляются как линейные комбинации старых: $z_{ij}^{'} = \sum\limits_{k=1}^{n} x_{ik}^{'} u_{kj}$.

<br>Если $m$ признаков могут быть выбраны в качестве базиса, а остальные линейное (или почти линейно) через них выражаются, то это будет хорошо видно на спектре (множестве с.зн. матрицы).
<br>Пусть $\lambda_1 \geq \ldots \geq \lambda_n \geq 0$. 
<br>*Эффективная размерность выборки* - это наименьшее целом $m$, при котором
$$E_m = \frac{\left\lVert G U^{T} - F \right\rVert^{2}}{\left\lVert F \right\rVert^{2}} = \frac{\lambda_{m+1} + \cdots + \lambda_n}{\lambda_1 + \cdots + \lambda_n} \leq \epsilon$$

<br>Так же обратим внимание, что SVD-разложение не единственное. Например, если $A$ квадратная и равна единичной матрице, то подойдет любое разложение вида $A = U E U^{T}$, где $U$ - произвольная ортогональная матрица. В данном случае эффект связан с тем, что все сингулярные значения одинаковые. Но, даже если сингулярные значения разные, всегда можно домножать соответствующие столбцы $U$ и $V$ на $-1$, например: 
$$\left( u_1 | \ldots | u_m \right) \Sigma \left( v_1 | \ldots | v_n \right)^{T} = A = 
\left( -u_1 | \ldots | u_m \right) \Sigma \left( -v_1 | \ldots | v_n \right)^{T}$$
Но по модулю вариантов домножения на много $-1$, в случае с различными собственными значениями, разложение определено однозначно. 
<br>На самом деле, достаточно, чтобы $\lambda_m$ и $\lambda_{m+1}$ были различны.

Последний пример так же показывает, что между столбцами $U$ и $V$ есть некоторое условие согласованности. Из-за неоднозначности в разложении надо аккуратно искать все его компоненты: если найденно что-то, например, матрицы $U$ и $\Sigma$, то матрицу $V$ надо искать не абы каким методом.
<br>---

<br>**Еще про SVD и PCA**.

Допустим, что у нас задан случайный вектор $\xi \in {\rm I\!R}^{n}$ и мы для него намерили независимо несколько семплов $x_1, \ldots, x_l \in {\rm I\!R}^{n}$. Первым делом можем оценить математическое ожидание и матрицу ковариации для $\xi$:
$$ {\rm I\!E}\xi = \overline{x} = \frac{x_1 + \cdots + x_l}{l}, \quad \Sigma_0 = \frac{1}{l-1} \left(x_i - \overline{x} \left) \right(x_i - \overline{x} \right)^{t}$$
Составим матрицу семплов $X = \left( x_1 | \ldots | x_l \right)$ и сдвинем каждый семпл на выборочное мат ожидание $Y = X - (\overline{x} | \ldots | \overline{x})$.

Что будет, если применить к данной матрице SVD? Алгоритм SVD начинается с того, что надо диагонализировать матрицу $Y Y^{T}$:
$\ Y Y^{t} = (x_1 - \overline{x})(x_1 - \overline{x}^{t} + \cdots + (x_l - \overline{x})(x_l - \overline{x})^{t}$, что с точностью до коэффициента совпадает с матрицей выборочной ковариации. Значит, диагонализация выборочной ковариации - первый шаг к SVD для $Y$. Интересненько. SVD для Y: $Y = U D V^{T}$.

<br>Но продолжим. 
<br>На SVD можно смотреть вот как: $D = U^{T} Y V$. Т.е. мы стартовали с матрицы из несмещенных семплов Y и теперь с помощью матрицы $U$ пытаемся поменять координаты в пространстве ${\rm I\!R}^{n}$, где живут наши семплы, а с помощью матрицы $V$ пытаемся комбинировать наши семплы между собой. Зачастую операция комбинирования семплов не очень физически/геометрически осмысленна, поэтому давайте ее проигнорируем и рассмотрим равенство $U^{T} Y = D V^{T}$. Тогда вот какой смысл у написанного. Мы стартовали с Y. Потом выбрали для выборочной матрицы ковариации $\Sigma_0$ с.в-ы ортонормированные и сложили их в матрицу $U = \left( u_1 | \ldots | u_n \right)$. Далее привели матрицу $Sigma_0$ к главным осям, т.е. сделали замену стандартного базиса на базис $u_1, \ldots, u_n$. При этом координаты наших семплов как раз изменятся по правило $ Y \to U^{T} Y = D V^{T} $. Если расписать это равенство в виде матриц, то увидим, что у ортогональной матрицы $V^{T}$ отрезали верхнюю часть. Так как матрица $V$ была ортогональна, то интуитивно, ее координаты вносят одинаковый вклад в итоговую матрицу. Однако, после этого мы каждую координату домножили на весовой коэффициент $\lambda_j$, а какие-то из последних даже на $0$. Таким образом после смены координат для несмещенных семплов $Y \to U^{T} Y$ мы отсортировали координаты по убыванию по их важности, где $\lambda_j$ означает вес важности координаты. При этом мы видим, если расписать в матричном виде, что последние координаты мы вообще можем проигнорировать, ибо они стали нулевыми. Кромет ого, мы еще можем проигнорировать координаты с малыми $\lambda_j$.

По другому еще можно сказать так. Мы нашли, что все наши семплы жили в подпространстве ${\rm I\!R}^{m}$ и после поворота пространства, отрезав лишние координаты, мы можем считать, что наши семплы имеют определенный специфичный вид: $D V^{T}$, где $D \in M_{m, m}, \ V \in M_{n, m}$.

<br>И последнее.
<br>Пусть $F \in M_{n, m}$ и ей в соответствие поставлен линейный оператор, также обозначаемый $F$. По SVD: $F = U \Lambda V^{T}$ -- это можно переформулировать в геометрических терминах. Линейный оператор, отобрадающий элементы пространства ${\rm I\!R}^{n}$ в элементы пространства ${\rm I\!R}^{m}$ представим в виде последовательно выполняемых линейных операций *вращения, растяжения и вращения*. Число ненулевых элементов на диагонали матрицы $\Lambda$ есть фактическая размерность матрицы $F$.

При этом по растяжением здесь имеется в виду диагональный оператор -- растяжение вдоль стандартного базиса (самосопряженный - вдоль любогго ортонормированного), причем каждую ось тянем со своим коэффициентом. SVD выделяет взаимосвязанные оси и коэффициенты их зависимостей друг с другом. При этом оси выделены явно в виде столбцов $U$ и $V$. А сингулярные значения матрицы $F$ - длины осей эллипсоида, заданного множеством $\{ A x = \left\lVert x \right\rVert^{2} \}$. 
<br>*Если просто раскладывать на вращение-растяжение, то это полярное разложение, и оно только для квадратных (в отличие от SVD) матриц.*

---

За 5 минут очень легко нашелся пакет в R'е для зафилливания библиотечным путем через SVD, а на Python довольно грустно, кажется. <br>Поэтому переключимся временно на язык R.

<br>Здесь подключим зафиллинный уже train, а код на R'е будет в отдельном файле.

In [ ]:
train.to_csv(r'\Data\missing_train.csv',
             index=False)

train_filled = pd.read_csv(r'\Data\filled_train.csv')

Так, ладно. У меня проблема с пакетами: то моя версия слишком старая, то теперь слишком новая, то хочет выделить 21000 Gb памяти. Вернемся в Python..

---

In [5]:
train_svd = train.copy()
test_svd = test.copy()
sorted(train.columns) == sorted(train_svd.columns)

True

Начнем с выкидывания полностью пустых колонок и зафилливания mean'ом (или median) пропусков.

In [6]:
mask = train_svd.isnull().sum() == 8000
cols_full_na = list(train_svd.columns[mask])

mask = test_svd.isnull().sum() == 8000
cols_full_na_test = list(test_svd.columns[mask])

train_svd = train_svd.drop(cols_full_na, axis=1)
train_svd.shape

(8000, 1188)

In [7]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
num_cols = train_svd.select_dtypes(include=numerics).columns
train_svd[num_cols] = train_svd[num_cols].fillna(0)

str_cols = train_svd.select_dtypes(include='object').columns
train_svd[str_cols] = train_svd[str_cols].fillna('')

train_svd.head()

,release,n_0000,n_0001,n_0002,n_0003,n_0004,n_0005,n_0006,n_0007,n_0009,...,c_1366,c_1367,c_1369,c_1370,c_1372,c_1373,c_1374,c_1375,c_1376,c_1377
0,a,0.0,0.0,0.025449,0.0,0.0,0.368421,0.0,0.0,0.0,...,,,,,a,,q,,,
1,a,0.0,0.0,0.031297,0.0,0.0,0.315789,0.0,0.0,0.0,...,,,,a,a,,,,,
2,a,0.0,0.0,0.024475,0.0,0.0,0.342105,0.0,0.0,0.0,...,,,,a,a,,b,,,
3,a,0.0,0.0,0.041694,0.0,0.0,0.447368,0.0,0.0,0.0,...,,,,,a,,,,,
4,c,0.0,0.0,0.038120,0.0,0.0,0.315789,0.0,0.0,0.0,...,,,,b,a,,a,,,


In [8]:
train_svd_ohe = OneHotEncoder(handle_unknown="ignore").fit_transform(train_svd[str_cols])
train_svd_ohe = pd.DataFrame.sparse.from_spmatrix(train_svd_ohe)

train_svd = train_svd.select_dtypes(include=numerics)
train_svd = pd.concat([train_svd, train_svd_ohe], axis=1, join="inner")
train_svd.shape

(8000, 6491)

Ищем SVD для нашей почищенной матрицы.

In [9]:
import scipy as sp

In [10]:
U, s, Vh = sp.linalg.svd(train_svd)
U.shape

(8000, 8000)

In [11]:
Sigma = np.zeros((train_svd.shape[0], train_svd.shape[1]))

Sigma[:train_svd.shape[1], :train_svd.shape[1]] = np.diag(s)

train_svd_matr = U.dot(Sigma.dot(Vh))
print(train_svd_matr)

[[ 3.43590445e-16 -7.11695179e-14  2.54494260e-02 ...  1.00000000e+00
   8.89489194e-15  4.26305066e-15]
 [ 8.30634389e-16 -2.98741615e-14  3.12973793e-02 ...  1.00000000e+00
   1.58788238e-15 -3.76034464e-15]
 [ 2.04128164e-16 -1.08820965e-14  2.44747672e-02 ...  1.00000000e+00
   9.19667717e-16 -6.55122386e-15]
 ...
 [-5.64327231e-17  9.04761905e-01  3.34632878e-02 ...  1.00000000e+00
   1.35151604e-14  1.04135494e-14]
 [-4.61243321e-16 -1.47268740e-14  4.71085120e-02 ...  1.00000000e+00
   2.86369224e-15  2.25538028e-15]
 [ 3.02397538e-16  7.61904762e-01  4.94910115e-02 ...  1.00000000e+00
   1.93411548e-14  2.03403284e-14]]


In [12]:
train_svd.head(3)

,n_0000,n_0001,n_0002,n_0003,n_0004,n_0005,n_0006,n_0007,n_0009,n_0010,...,6204,6205,6206,6207,6208,6209,6210,6211,6212,6213
0,0.0,0.0,0.025449,0.0,0.0,0.368421,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.031297,0.0,0.0,0.315789,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.024475,0.0,0.0,0.342105,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


Чорт, ```sp.linalg```, ```np.linalg``` не вычисляют SVD на размерности ниже.
<br>sklearn: 'Here we go again'.

In [13]:
from sklearn.decomposition import TruncatedSVD

In [14]:
## Тут, по сути, эксперимент, на практике, понятно, n_components определяются, например, через GridSearchCV
svd = TruncatedSVD(n_components=40, n_iter=10, random_state=42)  
train_svd = svd.fit_transform(train_svd)

train_svd

array([[ 3.07788041e+02, -1.72553499e+00, -1.77310983e+01, ...,
         2.85277375e+00,  9.37020409e-01, -1.96549767e+00],
       [ 3.00108913e+02,  1.49057653e+01, -2.86652132e+01, ...,
         7.33324297e-01, -6.27538923e+00,  5.25523910e+00],
       [ 1.38936860e+02,  3.35994699e+01, -1.44792801e+01, ...,
        -4.33372406e+00, -1.89300538e+00,  2.47522159e-01],
       ...,
       [ 3.73760917e+02, -6.44853793e+00, -2.61903158e+01, ...,
         2.96505670e+00, -4.79434516e-01,  1.61803180e+00],
       [ 8.05934310e+01,  7.31351717e+01,  4.81905283e+01, ...,
        -1.31522148e+00,  2.68243567e-01,  2.84171538e+00],
       [ 1.36445394e+02,  5.54581119e+01, -3.31549085e+01, ...,
        -9.06556466e-01,  5.77548873e-01,  5.32782783e-01]])

In [15]:
train_svd.shape

(8000, 40)

По всей видимости, нам вернулась разреженная матрица.

---

## Model I.4

In [16]:
lr = LogisticRegression(
    solver='lbfgs',
    random_state=42,
    n_jobs=-1
)
cbc = CatBoostClassifier(verbose=0)

models = {
    'LR': lr,
    'CB': cbc
}
predicts = []

In [18]:
for m in range(0, len(models)):
    model = list(models.values())[m]
    model_name = list(models)[m]
        
    t0 = datetime.utcnow()
    
    for K in range(0, labels.shape[1]):
        cv_values = cross_val_score(
            model,
            train_svd,
            labels.iloc[:, K],
            cv=4,
            scoring='neg_log_loss'
        )
        
        predicts.append(-cv_values.max())
        print(model_name + " " + str(K) + " test logloss = %.4f" % predicts[K])
            
    t1 = datetime.utcnow()
    print()
    print(model_name + " took {}".format(t1 - t0))
    
    print('---------------')
    print(model_name + ' test multiclass LogLoss = %.4f' % (sum(predicts) / len(predicts)))
    print('\n')
    predicts.clear()

LR 0 test logloss = 0.5916
LR 1 test logloss = 0.5818
LR 2 test logloss = 0.5507
LR 3 test logloss = 0.0503
LR 4 test logloss = 0.1913
LR 5 test logloss = 0.1159
LR 6 test logloss = 0.1916
LR 7 test logloss = 0.5536
LR 8 test logloss = 0.0744
LR 9 test logloss = 0.3803
LR 10 test logloss = 0.4892
LR 11 test logloss = 0.3149
LR 12 test logloss = 0.2751
LR 13 test logloss = 0.3620

LR took 0:00:48.983791
---------------
LR test multiclass LogLoss = 0.3373


CB 0 test logloss = 0.5591
CB 1 test logloss = 0.5821
CB 2 test logloss = 0.5504
CB 3 test logloss = 0.0538
CB 4 test logloss = 0.1963
CB 5 test logloss = 0.1204
CB 6 test logloss = 0.1949
CB 7 test logloss = 0.5427
CB 8 test logloss = 0.0797
CB 9 test logloss = 0.3702
CB 10 test logloss = 0.4908
CB 11 test logloss = 0.3025
CB 12 test logloss = 0.2671
CB 13 test logloss = 0.3700

CB took 0:05:00.168871
---------------
CB test multiclass LogLoss = 0.3343




Лучшее было:
<br>LR test multiclass LogLoss = 0.2792
<br>CB test multiclass LogLoss = 0.2505

Возможно, если поподбирать параметры, произойдет какое-то улучшение, но не понятно, существенное ли.
<br>Забавно, что в данном случае LR == CB, видимо, слишком переупростили данные.

---

**Еще чуть-чуть в глубь теории SVD**

Вкратце еще раз, как работает низкоранговое приближение $\left\lVert A - B \right\rVert^{2} \rightarrow \min\limits_{B}$.
<br>Сначала находим через SVD $A = U S V^{T}$. Потом надо найти матрицу $B$ ранга $m \leq n$, где $rk A = n$. Тогда в матрице $S$ останутся только первые наибольшие $m$ сингулярных значений, а остальные занулятся. Т.е. получу $S_m$. В итоге $B = U S_m V^{T}$. И если $A \in M_{p, q}$, то B тоже будет $\in M_{p, q}$.

<br>Возвращаясь к тому, что я хочу сделать: а именно, зафиллить miss'ы у $A$. Т.е. в теории хочу получить такую матрицу $B$, что где у $A$ miss'ов нет, то значения $A$ и $B$ совпадают, а где miss'ы есть, то значения из $B$ подставить взамен miss'ов $A$. 
<br>Но проблема в том, что низкоранговое SVD дает мне какую-то матрицу $B$, в том плане, что на ней лишь достигается $\min \left\lVert A - B \right\rVert^{2}$, но значения вообще могут со значениями $A$ не пересекаться. Информация из $A$ сохраняется только такая, "спектральная". В итоге это дело может невообразимо изменить исходное признаковое пространство. Поэтому, например, для рекомендация нам все равно, если мы от $A$ ни оставили ни одного исходного значения на том же месте, но для заполнения miss'ов нам абсолютно не все равно.

<br>Это я к чему. ЧЕРЕЗ ПРОСТО SVD MISS'Ы НЕ ЗАПОЛНИТЬ.

--
<br>Вообще говоря, возвращаясь к теории, при различных сингулярных (даже только при $m$, $m+1$) значения низкоранговое приближение почти единственное. 

Но ситуацию это не спасает по той причине, что SVD НЕ может работать с матрицами, содержащими NAs. В итоге, мы в лучше случае восстановим лишь почти что матрицу $\hat{A}$, которую уже заполнили mean'ми. Но нам нужно ее заполнить, а не восстановить mean'ированную с отличием лишь в знаках.
<br>--

---

## Gaps filling: part V

### Alternating Least Square

Хотим провести для матрицы $A$ разложение (matrix factorization) через матрицу $X$ или низкоранговое приближение $\left\lVert A - X \right\rVert_{*}^{*} \rightarrow \min\limits_{X}$, чтобы это минимизировало некоторый функционал потерь на матрицах $rk(X) \leq r \leq \min\{m, n\}$. Например: $J(X) = \left\lVert A - X \right\rVert_{F}^{2} + \lambda \left\lVert X \right\rVert_{F}^{2}, \ Q: {\rm I\!R}^{m \times n} \rightarrow {\rm I\!R}$.

<br>Для задачи о наилучшем приближении матрицы A ранга $r$ уже знаем, что SVD является лучшим приближнием A по нормам фробениусовой, $\ell_{2}$ и, <br>на самом деле, по любой унитарно-инвариантной норме.
<br>// *Норма на* $\mathbb{C}^{m \times n}$ *является унитарно-инвариантной, если $\left\lVert U A V \right\rVert$ = $\left\lVert A \right\rVert \ \ \forall \ \text{унитарных} \ U \in \mathbb{C}^{m \times m}, \ V \in \mathbb{C}^{n \times n} \ 
\& \ \forall A \in \mathbb{C}^{m \times n}$.*
<br>// *$U \in \mathbb{C}^{m \times n}$ - унитарная матрица, если $U U^{*} = E$ или (что тоже самое) $U^{-1} = U^{*} = \overline{U^{T}}$.*

Но на практике функционалы могут быть другими (и более сложными). Например: 
$$ Q(X) = \left\lVert P_{\Omega} \circ \left(A - X\right) \right\rVert_{F}^{2}, \quad  P_{\Omega} = I_{(i, j) \in \Omega} \quad \circ - \text{поэлементное произведение} $$
По сути, матрица $A$ известна только в некоторых позициях и там происходит умножение на $1$, а где не известна, умножение на $0$, в итоге фробениусовая норма не портится. Получаем задачу восстановления матрицы $A$ по наблюдаемым элементам: *matrix completion problem*.

Для $A = \begin{pmatrix} \text{NA} & \text{NA} & 5 \\ 3 & 2 & 2 \\ 5 & \text{NA} & 5 \end{pmatrix}$ имеем 
$P_{\Omega} = \begin{pmatrix} 0 & 0 & 1 \\ 1 & 1 & 1 \\ 1 & 0 & 1 \end{pmatrix}$.

<br>Вспомним, что для всякой матрицы $X$ порядка $m \times n$ ранга $r$ существует представление в виде произведения двух матриц $U$ и $V^T$, <br>где $U$ порядка $m \times r$, $V^{T}$ порядка $r \times n$, и их ранги равны $r$ (*скелетное разложение*).

В итоге, при выбранной норме функционала, приходим к следующей постановке задачи:
$$J \left( U, \ V \right) = \left\lVert A - U V^{T} \right\rVert_2 + \lambda \left( \left\lVert U \right\rVert_2 + \left\lVert V \right\rVert_2 \right) \rightarrow \min\limits_{U, \ V}$$
Первое слагаемое в чистом виде MSE - мера расстояния между исходной матрицей $A$ и ее аппроксимацией; второе - regularization term.
<br>Первое слагаемое есть функция затрат (потерь) $Q$: 
$$Q \left( U, \ V \right) = \left\lVert A - U V^{T} \right\rVert_2 = \sum\limits_{i, j} \left( A_{ij} - U_i V_j^T \right)$$
Такой объект не является выпуклым из-за члена $U_i V_j^{T}$; даже больше, такая задача является NP-сложной. Здесь можно использовать градиентный спуск в качестве приблизительного подхода, однако он оказывается медленным и требует большого количества итераций.
<br>Обратим внимание, что если мы зафиксируем набор переменных $V$ и будем рассматривать их как константы, то функционал будет выпуклой функцией $U$ и наоборот. Т.е. будем получать простую задача линейной регрессии, которая решается по методу наименьших квадратов (ordinary least squares). 
<br>// *Для $\left\lVert y - X \beta \right\rVert, \ y, \ X \ \text{- fix}$,  имеем: $\beta = \left(X^{T} X\right)^{-1} X^{T} y$.*

<br>**Идея**: по сути, ALS делает это же, но только в несколько этапов. Такой вот двухшаговый итерационный процесс: сначала фиксируется $V$ и решается задача оптимизации по $U$, затем фиксируется найденное $U$ и решается задача по $V$. Так как решение МНК (OLS) единственное и гарантирует минимум MSE, на каждом шаге функционал качества $J$ может либо убывать, либо оставаться неизменным, но никогда возрастать. Чередование двух шагов гарантирует не увеличение функции потерь $Q$, пока не сойдется, но, правда, только к локальному минимуму. Все это также очень зависит от начальных приближений $U_0$, $V_0$ в этом итерационном процессе.

<br>---

<br>**Алгоритм (ALS vanilla; 1994, Paatero, Tapper)**:
<br>Пусть $U_0, V_0$ - начальное приближение (например, случайные матрицы). Тогда 
$$U_{k+1} = arg\,\min_{U} Q(UV_{k}^{T}) \\
V_{k+1} = arg\,\min_{V} Q(U_{k+1} V^{T})$$

*Замечание*: минимизируя эти функционалы по $U$ и по $V$, может так произойти (особенно, если решение имеет rk $< r$), что столбцы $U$ или $V$ будут становиться все более и более мультиколлинеарными, что будет приводить к проблемам численной оптимизации.

Давайте дополнительно хотеть ортоганолизировать столбцы $U$ и $V$, чтобы не возникало мультиколлинеарности.

<br>**Алгоритм (ALS with orthogonalization)**:
<br>Пусть $U_0, V_0$ - начальное приближение (например, случайные матрицы). Тогда 
$$\hat{U}_{k+1} = arg\,\min_{U} Q(U V_k^T) \\
\hat{U}_{k+1} = Q_{k+1} R_{k+1} \quad \text{// QR-разложение, ортогонализуем //} \\
\hat{V}_{k+1} = arg\,\min_{V} Q(Q_{k+1} R_{k+1} V^T) = arg\,\min_{V} Q(Q_{k+1} \hat{V}^{T}) \\
\hat{V}_{k+1} = Q^{k+1} R^{k+1}$$ 
В итоге имеем:
$$U_{k+1} = Q_{k+1} \left(R^{k+1}\right)^{T} \\
V_{k+1} = Q^{k+1}$$
<br>При QR-разложении в вещественном случае матрица $Q$ будет ортогональной.

<br>Вообще говоря, еще у нас есть член регуляризации. Но его на каждом шаге можно разбить на подслагаемые и решать задачу оптимизации относительно новых функций потерь.

**Алгоритм (ALS with regularization)**:
<br>Пусть $U_0, V_0$ - начальное приближение (например, случайные матрицы). Тогда 
$$\forall U_i \ \ J(U_i) = \left\lVert A_i - U_i V^{T} \right\rVert_2 + \lambda \left\lVert U_i \right\rVert_2 \\
\forall V_j \ \ J(V_j) = \left\lVert A_j - U V_{j}^{T} \right\rVert_2 + \lambda \left\lVert V_j \right\rVert_2$$
По OLS имеем:
$$U_i^* = \left( V^T V + \lambda E \right)^{-1} V^{T} A_{i} \\
V_j^* = \left( U^T U + \lambda E \right)^{-1} U^{T} A_{j}$$

*Замечание*: так как $U_i^*$ ($V_j^*$) не зависит напрямую (но через $V$ ($U$)) от $U_{j \neq i}$ ($V_{i \neq j}$), то этот процесс распараллеливается.
<br>---

<br>**QR-разложение**:
<br>Матрица $A$ размера $n \times n$ с комплексными элементами может быть представлена в виде:
$$A = Q R$$
где $Q$ — унитарная матрица размера $n \times n$, а $R$ — верхнетреугольная матрица размера $n \times n$.

*Замечания*: 
<br>$1$. Если $A$ - квадратная невырожденная матрица, то существует единственное QR-разложение, если наложить дополнительное условие, что элементы на диагонали матрицы $R$ должны быть положительными вещественными числами.
<br>$2$. Если у $A$ первые $k$ столбцов линейно независимы, тогда первые $k$ столбцов матрицы $Q$ образуют ортогональный базис для этих столбцов матрицы $A$. 
<br>$3$. Тот факт, что любой столбец $K$ из А зависит только от первых $k$ столбцов $Q$ отвечает за треугольную форму $R$.

<br>---

<br>Когда мы хотим заполнить пропущенные значения при помощи матричного разложения, следует изменить функционал потерь:
$$\left\lVert P_{\Omega} \circ \left(A - U V^T\right) \right\rVert_{F}^{2} + \lambda \left( \left\lVert U \right\rVert_{F}^{2} + \left\lVert V \right\rVert_{F}^{2} \right) = \sum\limits_{i, j} p_{i j} \left(A_{i j} - U_i V_j^T \right)^{2} + \lambda \left( \left\lVert U \right\rVert_{F}^{2} + \left\lVert V \right\rVert_{F}^{2} \right)$$
где $p_{i j} = I_{A_{i j} - \text{известно}}$. Тогда решением МНК будет:
$$U_i^* = \left( V^T p_i V + \lambda E \right)^{-1} V^{T} p_i A_{i} \\
V_j^* = \left( U^T p_j U + \lambda E \right)^{-1} U^{T} p_j A_{j}$$

$P_{\Omega}$ не зануляет неизвестные элементы $A$. Мы лишь говорим, что при подсчете фробениусовой нормы, вклад от неизвестных элементов матрицы $A$ в эту норму будет нулевой. Если хотим делать малоранговое приближение, то на матрицу $X$ ($= U V^T$) нужно дополнительно накинуть ограничение (условие) малоранговости.
<br>---

<br>**Еще про ALS**.

При восстановлении пропущенных значений мы так же можем использовать не только матрциу $P_{\Omega}$, но и добавить новую матрицу $C_{\Omega}$. 
<br>Пусть с этого момента $p_{i j}$ будет называться *предпочтением* (*preference*), а $c_{i j}$ - *уверенностью* (*confidence*). 
<br>Новое решение состоит в том, чтобы объединить предпочтение (p) для элемента с уверенностью (c), которая у нас есть для этого предпочтения. <br>Мы начинаем с пропущенных значений как отрицательного предпочтения с низким значением достоверности и существующих значений как положительного предпочтения, но с высоким значением достоверности. Для задач коллаборативной фильтрации можно использовать что-то вроде количества воспроизведений, времени, проведенного на странице или какой-либо другой формы взаимодействия, в качестве основы для расчета уверенности.
$$p_{i j} = I_{a_{i j} \ известен} \\
c_{i j} = 1 + \alpha p_{i j}$$

Здесь доверие рассчитывается с использованием величины $p$ (данные обратной связи), что дает нам большую уверенность, чем больше раз, <br>в случае коллаборативной фильтрации (как пример), пользователь играл, просматривал или щелкал элемент. Скорость увеличения нашей уверенности задается линейным масштабным коэффициентом $\alpha$. Мы также добавляем $1$, чтобы получить минимальную уверенность, даже если $\alpha \times p$ равно нулю.
<br>Это также означает, что даже если у нас будет только одно взаимодействие между пользователем и элементом, достоверность будет выше, чем достоверность неизвестных данных с учетом значения $\alpha$. В [статье](http://yifanhu.net/PUB/cf.pdf) было обнаруженно, что $\alpha = 40$ работает хорошо.

<br>Теперь цель состоит в том, чтобы найти вектор $\forall \, U_i$ и $\forall \, V_j$ в размерах функций, что означает, что мы хотим минимизировать следующую функцию потерь:
$$\min_{U*, \ V*} \sum\limits_{i j} c_{i j} \left( p_{i j} - U_i V_j^{T} \right)^2 + \lambda (\left\lVert U_i \right\rVert_2 + \left\lVert V_j \right\rVert_2)$$
Как отмечается в этой же статье, если мы зафиксируем векторы $U$ или векторы $V$, то сможем вычислить *глобальный* минимум. Производная от приведенного выше уравнения дает нам следующее уравнение для минимизации потерь:
$$U_i = \left( V^{T} C^{i} V + \lambda E \right)^{-1} V^{T} C^{i} p(i) \\
V_j = \left( U^{T} C^{j} U + \lambda E \right)^{-1} U^{T} C^{j} p(j)$$
Еще один шаг заключается в том, что, понимая, что произведение $V^{T}$, $C^i$ и $V$ можно разделить следующим образом:
$$V^{T} C^{i} V = V^{T} V + V^{T} \left( C^{i} - E \right) V$$
Теперь имеем $V^T V$ и $U^{T} U$, независимые от $i$ и $j$, что означает, что можно предварительно вычислить их и сделать вычисления значительно менее интенсивными. Имея это в виду, окончательные уравнения:
$$U_i = \left( V^T V + V^{T} \left( C^i - E \right) V + \lambda E \right)^{-1} V^{T} C^i p(i) \\
V_j = \left( U^T U + U^{T} \left( C^j - E \right) U + \lambda E \right)^{-1} U^{T} C^j p(j)$$

---

Прежде чем что-то делать, проверим, совпадают ли размерности столбцов в наших данных.

In [19]:
print(train.shape[1] == test.shape[1])

True


Проверим, совпадают ли размерности числовых и категориальных признаков.

In [20]:
numeric_train_cols = list(train.select_dtypes(include=numerics).columns)
string_train_cols = list(train.select_dtypes(include='object').columns)

numeric_test_cols = list(test.select_dtypes(include=numerics).columns)
string_test_cols = list(test.select_dtypes(include='object').columns)

print(len(numeric_train_cols) == len(numeric_test_cols), 
      len(string_train_cols) == len(string_test_cols))

True True


Все хорошо. Можно приступать к тюнингу данных.

In [21]:
train_als = train.copy()
test_als = test.copy()
sorted(train.columns) == sorted(train_als.columns)

True

In [22]:
mask = train_als.isnull().sum() == 8000
cols_full_na = list(train_als.columns[mask])

mask = test_als.isnull().sum() == 8000
cols_full_na_test = list(test_als.columns[mask])

train_als = train_als.drop(cols_full_na, axis=1)
train_als.shape

(8000, 1188)

In [23]:
train_als[string_train_cols] = train_als[string_train_cols].fillna('')

train_als.head()

,release,n_0000,n_0001,n_0002,n_0003,n_0004,n_0005,n_0006,n_0007,n_0009,...,c_1366,c_1367,c_1369,c_1370,c_1372,c_1373,c_1374,c_1375,c_1376,c_1377
0,a,NaN,NaN,0.025449,NaN,NaN,0.368421,NaN,NaN,NaN,...,,,,,a,,q,,,
1,a,NaN,NaN,0.031297,NaN,NaN,0.315789,NaN,NaN,NaN,...,,,,a,a,,,,,
2,a,NaN,NaN,0.024475,NaN,NaN,0.342105,NaN,NaN,NaN,...,,,,a,a,,b,,,
3,a,NaN,NaN,0.041694,NaN,NaN,0.447368,NaN,NaN,NaN,...,,,,,a,,,,,
4,c,NaN,NaN,0.038120,NaN,NaN,0.315789,NaN,NaN,NaN,...,,,,b,a,,a,,,


In [24]:
train_als_ohe = OneHotEncoder(handle_unknown="ignore").fit_transform(train_als[string_train_cols])
train_als_ohe = pd.DataFrame.sparse.from_spmatrix(train_als_ohe)

train_als = train_als.select_dtypes(include=numerics)
train_als = pd.concat([train_als, train_als_ohe], axis=1, join="inner")
train_als.shape

(8000, 6491)

Ищем ALS для нашей обработанной матрицы.
<br>// Хотя уже не хорошо, что categorial признаки пришлось зафиллить, иначе OHE не работает.

In [25]:
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix    ## - support efficient modification; stores a list of (row, column, value) tuples
from scipy.sparse import csr_matrix    ## - support efficient access and matrix operations; represents a matrix M by three 
                                       ##   (one-dimensional) arrays, that respectively contain nonzero values, 
                                       ##   the extents of rows, and column indices

In [26]:
df_coo = coo_matrix(train_als.values)
df_coo

<8000x6491 sparse matrix of type '<class 'numpy.float64'>'
	with 9443612 stored elements in COOrdinate format>

In [27]:
# initialize a model
model = AlternatingLeastSquares(factors=40, iterations=100, calculate_training_loss=True) ## loss=nan всегда отображ здесь

# train the model on a sparse matrix of item/user/confidence weights
start = datetime.utcnow()
model.fit(df_coo)  
print(f"total time: {datetime.utcnow() - start}")

ModelFitError: NaN encountered in factors

Я просто поражаюсь, что ALS'у уже ~30 лет, проблема miss'ов в рекомендациях давнишняя, но они не смогли реализовать нормально алгоритм заполнения miss'ов для заполнения miss'ов -- упал на miss'ах. Аплодирую стоя.

Охх, все беды от работы с чужим кодом..

In [28]:
train_als = train_als.fillna(0)
df_csr = csr_matrix(train_als.values)
df_csr

<8000x6491 sparse matrix of type '<class 'numpy.float64'>'
	with 7573912 stored elements in Compressed Sparse Row format>

In [29]:
# initialize a model
model = AlternatingLeastSquares(factors=40, iterations=100, calculate_training_loss=True) ## loss=nan всегда отобр здесь

# train the model on a sparse matrix of item/user/confidence weights
start = datetime.utcnow()
model.fit(df_csr)  
print(f"total time: {datetime.utcnow() - start}")


total time: 0:00:56.057166


Не знаю, как реализован als точно в этой библиотеке, но что сейчас, что до этого я игрался, loss ~ за итераций 10 доходит до локального экстремума и все. Это невероятно. Причем игрался я на очень классных матрицах, а loss у als сходился всегда довольно быстро к фигне какой-то в целом. Неужели за 30 лет никто не сделал норм либу по ALS..

Я, конечно не совсем права, раз в 200 итераций loss все же изменяется на целую тысячную!

In [30]:
pd.DataFrame.sparse.from_spmatrix(df_csr).head()

,0,1,2,3,4,5,6,7,8,9,...,6481,6482,6483,6484,6485,6486,6487,6488,6489,6490
0,0.0,0.0,0.025449,0.0,0.0,0.368421,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.031297,0.0,0.0,0.315789,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.024475,0.0,0.0,0.342105,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.041694,0.0,0.0,0.447368,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.038120,0.0,0.0,0.315789,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [31]:
print(model.item_factors.shape, model.user_factors.shape, sep='\n')

(8000, 40)
(6491, 40)


In [32]:
# recommend items for a user
df_csr = df_csr.T.tocsr()
df_csr_fitted = model.recommend(0, df_csr)
df_csr_fitted

[(7306, 0.010278553),
 (4967, 0.00992531),
 (7282, 0.009865671),
 (4806, 0.009752322),
 (6458, 0.009576814),
 (1257, 0.009567821),
 (5254, 0.009473771),
 (2329, 0.009408616),
 (2917, 0.009396792),
 (2984, 0.009393851)]

Это пичаль.

--

Еще я игрался с таким, приводя матрицу к 3м столбцам: ```items```, ```users```, ```rating```.
<br>Но прямо скажем, что какие-то танцы с бубнами (что для зафилливания мне приходится изменять полностью вид).

In [ ]:
df_train = pd.melt(train_als, ignore_index = False)
df_train['index'] = [i for i in range(0, train_als.shape[0])] * train_als.shape[1]
df_train.dropna()

# В конце нужно было бы сделать так
df_train_original = df_train.pivot(columns="variable", values="value")

Но какого-то фига, даже если захотеть пошаманить, этот ALS полностью изменял матрицу исходную (train_als) -- это вообще как? <br>Какой ALS они используют..

Вывод: писать ALS самому.

--

---